# Assignment 4: Sampling

##

In this assignment we are going to look into a practical scenario of sampling a signal. We will start with a quadrature-phase-shift-keying (QPSK) modulated signal. This is a common modulation technique to transmit digital data over analog channels, such as wireless radio or optical fibres. These sampling (and receiver) techniques, however, are also used when recording physical measurement data, such as in lock-in amplifiers. In both cases we try to sample as much relevant information as possible from our analog signal that we then can use within our digital signal processing system.

### You may need to rerun all cells at once some times. This is due to the randomly generated bit sequence.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

## QPSK

QPSK is a common modulation technique in communication technology to encode digital data. Feel free to look [here](https://nl.wikipedia.org/wiki/Phase_shift_keying) for more information, but QPSK itself is not relevant for the course or exam. In a nutshell, we take 2 bits (i.e. (1,0)) and encode them with a phase shift (i.e. 90° or 1+1j).

Here, we have a 100 kHz sine wave that we call the carrier. This we will modulate with our a lower frequency signal, our message (baseband). The message consists of 2 bits per symbol, at a rate of 5k symbols/s, leading to a bit rate of 10 Kb/s.
Please note that the bits are generated randomly. So each execution of the following cell will lead to a different sequence. You can execute the whole notebook multiple times to see different possible sequences.

This leads us to a 5 kHz modulated on top of a 100 kHz signal.
The following block is just a setup and you do not need to fill anything.

In [ ]:
sampling_rate = 1e6 # This is chosen very high to correctly show our signal as quasi-analog. It is not the tehcnically relevant sampling rate of the ADC we will talk about later
nyq = sampling_rate / 2
samples = 1024*100
carrier_omega = 2 * np.pi * 100e3
symbol_rate = 5e3

n_symbols = symbol_rate / sampling_rate * samples
samples_per_symbol = samples / n_symbols
bits = np.random.randint(0, 2, int(2*n_symbols))
bit_pairs = bits.reshape(-1, 2)
mapping = {
    (0, 0): 1 + 1j,
    (0, 1): -1 + 1j,
    (1, 1): -1 - 1j,
    (1, 0): 1 - 1j
}
symbols = np.array([mapping[tuple(b)] for b in bit_pairs])
baseband = np.repeat(symbols, samples_per_symbol)

print(f"Our first bit pairs are: {bit_pairs[:5]}")
print(f"Our first symbols are: {symbols[:5]}")

time = np.arange(samples) / sampling_rate
carrier = np.sin(carrier_omega * time)

carrier_i = np.cos(carrier_omega * time)
carrier_q = np.sin(carrier_omega * time)
qpsk_signal = np.real(baseband) * carrier_i - np.imag(baseband) * carrier_q

fig_mod, ax_mod = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_mod.plot(1e6*time, qpsk_signal)
ax_mod.set_xlabel("Time [μs]")
ax_mod.set_ylabel("Amplitude [a.u.]")
ax_mod.set_xlim([50,450])
ax_mod.set_title("Modulated Carrier")

## Task 1: Direct sampling

You will now need to sample this signal. In this case, since it already is digital, we are emulating the sampling process by reducing the sampling rate.

Your task is to choose the highest decimation factor (lowest sampling rate) that allows you to still see bit changes in the following plots. Run the notebook several times to see how it looks like for different symbol changes, or not symbol change at all.

In [ ]:
factor_direct = # Find the highest factor that allows you to see bit changes in symbol 1->2 and 2->3. The ADC sampling rate (so the effective sampling rate) is then sampling_rate/factor_direct.
print(f"Sampling rate for direct sampling: {sampling_rate/factor_direct}")

direct_sampled = sp.signal.decimate(qpsk_signal, factor_direct)
time_sampled_direct = np.arange(samples/factor_direct) / sampling_rate * factor_direct

fig_direct1, ax_direct1 = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_direct1.plot(1e6*time, qpsk_signal)
ax_direct1.plot(1e6*time_sampled_direct, direct_sampled)
ax_direct1.set_xlabel("Time [μs]")
ax_direct1.set_ylabel("Amplitude [a.u.]")
ax_direct1.set_xlim([150, 250])
ax_direct1.set_title("Sampled (Symbol 1->2)")

fig_direct2, ax_direct2 = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_direct2.plot(1e6*time, qpsk_signal)
ax_direct2.plot(1e6*time_sampled_direct, direct_sampled)
ax_direct2.set_xlabel("Time [μs]")
ax_direct2.set_ylabel("Amplitude [a.u.]")
ax_direct2.set_xlim([350, 450])
ax_direct2.set_title("Sampled (Symbol 2->3)")
ax_direct1.set_xlim([350, 450])
ax_direct1.set_title("Sampled (Symbol 2->3)")

### Questions:
- Look at the plots of the signal and how it changes based between the symbols. How (in what part of the carrier signal) is the information encoded? Rerun the notebook to see several possibilites.
  - YOUR ANSWER
- In this case we have two signals (carrier and message/baseband), which one is dictating the minimum sampling (Nyquist) rate needed? Why?
  - YOUR ANSWER


## Task 2: IQ - Sampling

Now we will try another sampling technique, called In-Phase/Quadrature sampling. You can also find it referred to as just quadrature sampling or direct conversion. Refer to the lecture on sampling for the concept.

Your tasks are:
- Calculate the I and Q components as referred to in the lecture
- Choose an appropriate lowpass cutoff frequency
- Find the minimum factor (sampling rate) to reconstruct the original message (bit sequence)
  - Run it multiple times to check if it works with different sequences.

If you can see something looking like a bit sequence (on/off sequence) in the first figure you are on the right way. Note that there can be some overshoot in the rectangles.

In [ ]:
# Get the I and Q components from qpsk_signal
i_component = # qpsk_signal * [...]
q_component = # qpsk_signal * [...]

# Now we need to lowpass
cutoff = # Your cutoff frequency (lowpass). 

sos = sp.signal.butter(3, cutoff/nyq, btype="lowpass", output="sos")
i_lowpassed = sp.signal.sosfilt(sos, i_component)
q_lowpassed = sp.signal.sosfilt(sos, q_component)

fig_demod, ax_demod = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_demod.plot(1e6*time, i_lowpassed, label="I")
ax_demod.plot(1e6*time, q_lowpassed, label="Q")
ax_demod.set_xlabel("Time [μs]")
ax_demod.set_ylabel("Amplitude [a.u.]")
ax_demod.set_xlim([0,1000])
ax_demod.set_title("IQ Demodulated")
ax_demod.legend()

# Now we emulate an ADC by downsampling our "analog" signal. Again choose the maximum factor (minimum sampling rate) to reconstruct the information.
factor_iq = # Your factor
print(f"Sampling rate for IQ sampling: {sampling_rate/factor_iq}")

i_sampled = sp.signal.decimate(i_lowpassed, factor_iq)
q_sampled = sp.signal.decimate(q_lowpassed, factor_iq)
time_sampled_iq= np.arange(samples/factor_iq) / sampling_rate * factor_iq

fig_sampled, ax_sampled = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_sampled.plot(1e6*time_sampled_iq, i_sampled, label="I")
ax_sampled.plot(1e6*time_sampled_iq, q_sampled, label="Q")
ax_sampled.set_xlabel("Time [μs]")
ax_sampled.set_ylabel("Amplitude [a.u.]")
ax_sampled.set_xlim([0,1000])
ax_sampled.set_title("Sampled")
ax_sampled.legend()

### Questions:
- What does the low-pass accomplish?
  - YOUR ANSWER
- What influence does the cutoff sfrequency have on the shape of the IQ demodulated signal?
  - YOUR ANSWER
- How (in what part of the signal) is the information now encoded in the IQ demodulated signal?
  - YOUR ANSWER
- What signal is now dictating the Nyquist rate?
  - YOUR ANSWER
- What is the advantage of IQ sampling, and what is the disadvantage?
  - YOUR ANSWER
